# PRACTICA GUIADA: Introducción a Machine Learning 1

Ahora revisaremos varios ejemplos simples de aplicación de métodos de aprendizaje supervisado y no supervisado

### Ejemplo de aprendizaje supervisado: Regresión lineal simple

Como ejemplo de este proceso, vamos a considerar una regresión lineal simple, es decir, el caso común de ajustar una línea a datos de la forma $(x, y)$. 

Vamos a utilizar el dataset de publicidad presentado en el manual 'An Introduction to Statistical Learning with Applications in R' de James, Witten, Hastie, Tibshirani (2013).

Primero importamos las librerías necesarias para trabajar con datos y visualizar:

In [4]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
%matplotlib inline 

Vamos a importar el dataset de publicidad para nuestro ejemplo de regresión:

In [2]:
from google.colab import files
uploaded = files.upload()

ImportError: No module named 'google.colab'

In [5]:
advertising = pd.read_csv('Advertising.csv', usecols=[1,2,3,4])
advertising.info()

FileNotFoundError: File b'Advertising.csv' does not exist

Vamos a observar las primeras 5 filas de nuestro dataset. 

Las primeras 3 columnas representan los gastos de publicidad en miles de USD en diferentes medios (TV, radio y diarios). Estas 3 columnas van a ser nuestras **features**.

La cuarta columna representa las ventas de un determinado producto en miles de unidades de la empresa que invierte en publicidad. Esta va a ser nuestra variable **target**.

In [0]:
advertising.head()

Visualizamos los datos haciendo un pairplot:

In [0]:
sns.pairplot(advertising);

Calculamos la matriz de correlación:

In [0]:
advertising.corr()

Como nuestro primer objetivo es hacer un modelo de regresión simple, es decir con una sola feature, vamos a elegir la variable TV, ya que es la que muestra una correlación mayor con nuestra variable objetivo.

Vamos a hacer un scatter plot entre TV y Sales. El gráfico también nos va a mostrar la recta que se obtiene con una regresión lineal simple:

In [0]:
sns.regplot(advertising.TV, advertising.Sales, order=1, ci=None,\
                                scatter_kws={'color':'r', 's':9})
plt.xlim(-10,310)
plt.ylim(bottom=0);

#### 1. Seleccionar una "clase de modelo"

En Scikit-Learn, cada clase de modelo se representa con una clase de Python. 

Entonces, por ejemplo, si queremos computar un modelo de regresión lineal simple, podemos importar la clase de regresión lineal de esta forma:

In [0]:
from sklearn.linear_model import LinearRegression

Notar que también existen otros modelos de regresión lineal más generales; Podés leer más acerca de ellos en la [documentación ``sklearn.linear_model``](http://Scikit-Learn.org/stable/modules/linear_model.html). 

#### 2. Elegir los hiperparámetros del modelo

Es importante destacar que *una clase de modelo no es lo mismo que una instancia de modelo*.

Una vez que hemos decidido nuestra clase de modelo, todavía tenemos que tomar algunas decisiones. Dependiendo de la clase de modelo con la que trabajemos, podríamos tener que responder a una o más preguntas como las siguientes:

- ¿Queremos incluir también un intercepto (intercept = True)?
- ¿Queremos que el modelo esté normalizado?
- ¿Queremos agregar features calculados a partir del input para darle mayor flexibilidad al modelo?
- ¿Qué grado de "regularización" vamos a querer usar en el modelo?

Estos son ejemplos de las importantes decisiones que deben hacerse **una vez que hemos seleccionado la clase de modelo a usar**.

Estas elecciones se representan frecuentemente como *hiperparámetros*, o parámetros que deben ser seteados antes de que el modelo sea ajustado a los datos. 

En Scikit-Learn, los hiperparámetros son elegidos como argumentos en la instanciación del modelo. Exploraremos cómo podemos justificar cuantitativamente la elección de hiperparámetros en las próximas clases.  

Para nuestro ejemplo de regresión lineal, podemos instanciar la clase ``LinearRegression`` y especificar que nos gustaría ajustar el intercepto usando el hiperparámetro ``fit_intercept``: 

In [0]:
model = LinearRegression(fit_intercept=True)
model

**Tener en cuenta**: cuando el modelo es instanciado, la única acción que sucede es almacenar los valores de estos valores de hiperparámetros.

En particular, todavía no hemos aplicado el modelo a ningún dato: la API de Scikit-Learn hace una distinción muy clara entre la *elección del modelo con sus hiperparámetros* y la *aplicación del modelo a los datos*. 

#### 3. Preparar los datos en una matriz de features y un vector de target

Previamente hemos hablado de la representación de datos de Scikit-Learn, la cual requiere una matriz de features de dos dimensiones y un vector target de una dimensión.

A continuación creamos la matriz de features y el vector target: 

In [0]:
# Creamos X e y

feature_cols = ['TV']
X = advertising[feature_cols]
y = advertising.Sales

print("Shape X:", X.shape)
print("Shape y:", y.shape)
print("Type X:", type(X))
print("Type y:", type(y))

#### 4. Split entre set de entrenamiento y de testeo

Nos gustaría evaluar el modelo en datos que no hayan sido usados en el entrenamiento, por lo tanto vamos a dividir los datos en un *training set* y un *testing set*.

Esto podría hacerse a mano, pero es más conveniente usar la función ``train_test_split``.

In [0]:
from sklearn.model_selection import train_test_split
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y,\
                                                random_state=1)

#### 4. Ajustar el modelo a los datos

Ahora es momento de aplicar nuestro modelo a los datos.
Esto puede hacerse con el método ``fit()`` de nuestra instancia de modelo.

In [0]:
model.fit(Xtrain, ytrain)

El método ``fit()`` realiza una secuencia de cómputos internos dependientes del modelo, y los resultados de estas operaciones son almacenadas en atributos específicos de la clase de modelo que el usuario luego puede explorar.

En Scikit-learn, por convención, todos los atributos que representan los parámetros de los modelos que fueron aprendidos durante el procesos de entrenamiento con ``fit()``, tienen `underscores` en sus nombres; por ejemplo en este modelo lineal, podemos observar el parámetro coef_ y el parámetro intercept_:

In [0]:
model.coef_

In [0]:
model.intercept_

Estos dos parámetros representan la pendiente y el intercepto del ajuste lineal simple a los datos. ¿Cómo interpretamos a estos valores?

Una pregunta que surge frecuentemente se relaciona con incertidumbre o incerteza (uncertainty) en estos parámetros internos del modelo. 

En general, Scikit-Learn no provee herramientas para obtener conclusiones del estado interno de los modelos: interpretar los parámetros de un modelo tiene mucho más que ver con una pregunta de *modelado estadístico* más que una pregunta de *machine learning*.

Machine learning en cambio se enfoca en la calidad con la cual el modelo *predice*.

Si te interesa investigar el significado de los parámetros de ajuste dentro del modelo, existen otras herramientas, incluyendo el paquete de python [Statsmodels](http://statsmodels.sourceforge.net/).

#### 5. Predecir etiquetas para datos desconocidos

Una vez que el modelo es entrenado, la principal tarea en el aprendizaje supervisado es evaluarlo en base a lo que dice acerca de nuevos datos que no fueron parte del **set de entrenamiento**. 

En Scikit-Learn, esto puede hacerse usando el método ``predict()``. 

Para ganar intuición, empecemos aplicando la forma manualmente:

In [0]:
# Aplicando la fórmula manualmente

test = 200

model.intercept_ + model.coef_*test

In [0]:
# usando el método del objeto
import numpy as np

test_sklearn = np.array(test).reshape(-1,1)

model.predict(test_sklearn)

Podemos calcular las predicción del modelo para las observaciones en el set de testeo usando el método ``predict()`` de la siguiente forma: 

In [0]:
ypred = model.predict(Xtest)

Finalmente, podemos evaluar la bondad de ajuste utilizando las siguientes métricas:

In [0]:
from sklearn import metrics
print ('MAE:', metrics.mean_absolute_error(ytest, ypred))
print ('MSE:', metrics.mean_squared_error(ytest, ypred))
print ('RMSE:', np.sqrt(metrics.mean_squared_error(ytest, ypred)))
print ('R2:', metrics.r2_score(ytest, ypred))

Donde:
    
** El error absoluto medio ** (MAE) es la media del valor absoluto de los errores:

$$ \frac 1n\sum_ {i = 1}^n |y_i-\hat{y}_i| $$

** Mean Squared Error ** (MSE) es la media de los errores al cuadrado:

$$ \frac 1n\sum_ {i = 1}^n(y_i- \hat{y}_i)^2 $$

** Error cuadrático medio raíz ** (RMSE) es la raíz cuadrada de la media de los errores al cuadrado:

$$ \sqrt{\frac 1n\sum_{i = 1}^n(y_i- \hat{y}_i)^2} $$

Comparando estas métricas:

- ** MAE **  es el error promedio.
- ** MSE **  "penaliza" errores grandes.
- ** RMSE **  es interpretable, tiene las mismas unidades  que la "y".
- ** $R^2$ ** es la proporción de la varianza total de $Y$ explicada por el modelo

Con excepción de R2, todas estas son ** funciones de pérdida **, porque queremos minimizarlas.

En la clases que vienen vamos a estudiar en profundidad el modelo de regresión lineal. Pasemos ahora a ver un ejemplo de clasificación. 

### Ejemplo de aprendizaje supervisado: Default de tarjeta de crédito

Veamos un modelo de clasificación. Este es otro tipo de problema de aprendizaje supervisado pero aquí la variable objetivo es categórica. Queremos predecir si una determinada persona entrará en default en el pago de su tarjeta de crédito, tomando como features su ingreso, la deuda en su tarjeta (balance) y si es estudiante o no.

Este dataset también lo tomamos del manual 'An Introduction to Statistical Learning with Applications in R' de James, Witten, Hastie, Tibshirani (2013)


In [0]:
# Importamos el dataset y visualizamos las primeras 5 filas:

df = pd.read_excel('Default.xlsx', usecols='B:E')

df.head()

In [0]:
default = pd.get_dummies(df.default, prefix='default', prefix_sep='_', drop_first=True)
student = pd.get_dummies(df.student, prefix='student', prefix_sep='_', drop_first=True)

In [0]:
default.head()

In [0]:
df = pd.concat([df, student, default], axis=1)
df.head()

In [0]:
# Creamos X e y

feature_cols = ['balance', 'income', 'student_Yes']
X = df[feature_cols]
y = df.default_Yes

print("Shape X:", X.shape)
print("Shape y:", y.shape)
print("Type X:", type(X))
print("Type y:", type(y))

Para esta tarea, usaremos un modelo generativo extremadamente simple conocido como Naive Bayes Gausiano, el cuál procede asumiendo que cada clase se construye a partir de una distribución Gausiana. Lo veremos en detalle más adelante en el curso.

Porque es muy rápido y no tiene hiperparámetros para elegir, Naive Bayes Gausiano es frecuentemente un buen modelo para usar como una clasificación baseline, antes de explorar si pueden encontrarse mejoras a través de modelos más sofisticados.

Vamos a generar nuestros datasets de entrenamiento y testeo usando la función ``train_test_split``.

In [0]:
from sklearn.model_selection import train_test_split
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y,
                                                random_state=1)

Con los datos preparados, podemos seguir nuestra receta para predecir las etiquetas:

In [0]:
from sklearn.naive_bayes import GaussianNB # 1. elegir la clase de modelo
model = GaussianNB()                       # 2. instanciar el modelo
model.fit(Xtrain, ytrain)                  # 3. ajustar el modelo a los datos
y_model = model.predict(Xtest)             # 4. predecir a partir de nuevos datos

Finalmente, podemos usar la función ``accuracy_score`` para estudiar la proporción de etiquetas predichas que coinciden con el valor de verdad correspondiente a esa observación.

In [0]:
from sklearn.metrics import accuracy_score
accuracy_score(ytest, y_model)

Con un accuracy del 97,4%, podemos ver que incluso este sencillo algoritmo de clasificación es efectivo para este dataset particular.

### Ejemplo de aprendizaje no supervisado: Dimensionalidad de Iris

Como un ejemplo de un problema de aprendizaje no supervisado, veamos cómo reducir la dimensionalidad de los datos del dataset Iris para poder visualizarlos más fácilmente.

El dataset consiste en 50 muestras de 3 especies de la flor iris, de las cuales disponemos 4 features: el largo y ancho del pétalo y del sépalo.

Por lo tanto, el dataset Iris es cuatridimensional: hay cuatro features medidas para cada observación (sample).

La tarea de reducción de la dimensionalidad es investigar si hay una representación apropiada de baja dimensionalidad que retiene las características esenciales del dataset original. 

Frecuentemente la reducción de la dimensionalidad se usa como una ayuda para visualizar datos: después de todo es mucho más fácil plotear datos en dos dimensiones que en cuatro o más dimensiones. 

En este ejemplo vamos a usar Principal Component Analysis (PCA), que es una técnica rápida de reducción lineal de la dimensionalidad. 
Vamos a pedirle al modelo que devuelva dos componentes, es decir, una representación bidimensional de los datos. 

Siguiendo la secuencia de pasos presentada previamente, tenemos:

In [0]:
# importamos el dataset de la libresría Seaborn:

iris = sns.load_dataset('iris')
iris.head()

In [0]:
# Generamos la matriz de las features:
X_iris = iris.drop('species', axis=1)
X_iris.species

In [0]:
# 1. Seleccionar la clase de modelo
from sklearn.decomposition import PCA  

# 2. Instanciar el modelo con hiperparámetros
model = PCA(n_components=2)            

# 3. Ajustar a los datos. Notar que no especificamos "y" 
model.fit(X_iris)                      

# 4. Transformar los datos a dos dimensiones
X_2D = model.transform(X_iris)         

Ahora vamos a plotear los resultados. Una forma rápida de hacer esto es insertar los resultados en el ``DataFrame`` original de Iris, y usar el método ``lmplot`` de Seaborn para mostrar los resultados:


In [0]:
iris['PCA1'] = X_2D[:, 0]
iris['PCA2'] = X_2D[:, 1]
sns.lmplot("PCA1", "PCA2", hue='species', data=iris, fit_reg=False);

In [0]:
iris.head()

Vemos que en la representación en dos dimensiones, las especies están relativamente bien separadas, incluso aunque el algoritmo PCA no tenía conocimiento de las etiquetas de las especies de flores! 

### Aprendizaje no supervisado: Clustering con Iris

Vamos a aplicar un algoritmo de clustering al dataset Iris.

Un algoritmo de clustering intenta encontrar grupos distintos sin tener referencias a etiquetas en los datos. 

Vamos a usar un método poderoso de clustering llamado Gaussian mixture model (GMM). 
Un GMM intenta modelar los datos como una colección de blobs Gausianos. 

Podemos ajustar el GMM de la siguiente forma:

In [0]:
# 1. Elegimos la clase de modelo
from sklearn.mixture import GaussianMixture

# 2. Instanciamos el modelo con sus hiperparámetros
model = GaussianMixture(n_components=3,
            covariance_type='full')  

# 3. Ajustamos a los datos. Notar que "y" no es especificada
model.fit(X_iris)                    

# 4. Determinamos las etiquetas de los clusters
y_gmm = model.predict(X_iris)       

Como antes, agregaremos las etiquetas de los clusters al ``DataFrame`` Iris y usaremos Seaborn para plotear los resultados:

In [0]:
iris['cluster'] = y_gmm
sns.lmplot("PCA1", "PCA2", data=iris, hue='species',
           col='cluster', fit_reg=False);

Al separar los datos por número de cluster, vemos exactamente cuán bien el algoritmo GMM ha recuperado la etiqueta subyacente: la especie *setosa* es separada perfectamente, aunque vemos una pequeña porción mezclada entre *versicolor* y *virginica*. 

Esto significa que incluso sin un experto que nos diga las etiquetas de las flores individuales, las medidas de estas observaciones son lo suficientemente distintas para que podamos identificar *automáticamente* la prsencia de estos diferentes grupos de especies con un simple algoritmo de clustering. 

Este tipo de algoritmo podría incluso darle a los expertos en la disciplina algunas pistas sobre las relaciones entre las muestras que están observando.

## En resumen

En esta sección hemos cubierto las características esenciales de la representación de datos en Scikit-Learn y la API de estimadores. 

Sin importar el tipo de estimador, el mismo patrón de importar/instanciar/fittear/predecir se mantiene en todos los casos. 

Armado con esta nueva información sobre la API de estimadores, vos podés explorar la documentación de Scikit-Learn y comenzar a probar varios modelos sobre tus datos. 

En la próxima clase, vamos a explorar el que tal vez es el tópico más importante en machine learning: cómo seleccionar y validar tu modelo. 